<a href="https://colab.research.google.com/github/praveen61204/HCLTECH/blob/master/assignment/paper1Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory

dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
path = tf.keras.utils.get_file("cats_and_dogs.zip", origin=dataset_url, extract=True)

train_dir = path.replace("cats_and_dogs.zip", "cats_and_dogs_filtered/train")
val_dir = path.replace("cats_and_dogs.zip", "cats_and_dogs_filtered/validation")
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

train_ds = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

val_ds = image_dataset_from_directory(
    val_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))

base_model = MobileNetV2(input_shape=IMG_SIZE + (3,),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

global_avg = layers.GlobalAveragePooling2D()(base_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(global_avg)

model = tf.keras.Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5)
base_model.trainable = True

for layer in base_model.layers[:-40]:
    layer.trainable = False                            #Layer is reference to the base_model.layers

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=5)

loss, acc = model.evaluate(val_ds)
print(f"Fine-tuned accuracy: {acc*100:.2f}%")

Found 3000 files belonging to 1 classes.
Found 3000 files belonging to 1 classes.
Epoch 1/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 43s 272ms/step - accuracy: 0.9864 - loss: 0.0676 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 2/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 87ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 6.7305e-04
Epoch 3/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 1.0000 - loss: 5.8113e-04 - val_accuracy: 1.0000 - val_loss: 4.3589e-04
Epoch 4/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 1.0000 - loss: 3.8028e-04 - val_accuracy: 1.0000 - val_loss: 3.0861e-04
Epoch 5/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 1.0000 - loss: 2.8264e-04 - val_accuracy: 1.0000 - val_loss: 2.3224e-04
Epoch 1/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 42s 249ms/step - accuracy: 1.0000 - loss: 9.3019e-05 - val_accuracy: 1.0000 - val_loss: 9.5043e-05
Epoch 2/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 1.0000 - loss: 2.9653e-05 - val_accuracy: 1.0000 - val_loss:

In [22]:
import torch
import cv2
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load YOLOv8
model=YOLO("yolov8n.pt")
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s')


img = "download.jpeg"
image = cv2.imread(img)
results = model(img)
result = results[0]

# Get annotated image (BGR)
annotated_img = result.plot()

# Convert to RGB
annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 8))
plt.imshow(annotated_img)
plt.axis("off")
plt.show()

for box in results[0].boxes:
    cls = int(box.cls[0])
    conf = float(box.conf[0])
    print(f"Detected: {model.names[cls]} (Confidence: {conf:.2f})")



image 1/1 /content/download.jpeg: 288x640 8 persons, 1 chair, 8.8ms
Speed: 1.2ms preprocess, 8.8ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)
Detected: person (Confidence: 0.83)
Detected: person (Confidence: 0.82)
Detected: person (Confidence: 0.80)
Detected: person (Confidence: 0.76)
Detected: person (Confidence: 0.75)
Detected: person (Confidence: 0.67)
Detected: person (Confidence: 0.49)
Detected: person (Confidence: 0.46)
Detected: chair (Confidence: 0.38)
